# Get Raw Images

This notebook will demonstrate how to search for a specific image in order to get its `sequence_id`.

Using a `sequence_id` we will create an `ObservationInfo` object for discovering information about the observation before showing how to download the image metadata and FITS files.

In [ ]:
import pandas as pd
from astropy.coordinates import SkyCoord

from panoptes.data.search import search_observations
from panoptes.data.observations import ObservationInfo

## Search for observations

Using a sky coordinate, search for all matching observations with a certain radius.

In [ ]:
search_coords = SkyCoord.from_name('M42')
search_coords

In [ ]:
search_results = search_observations(
    coords=search_coords,
    radius=10,
    min_num_images=5,
    unit_id='PAN019'
)

print(f'Found {len(search_results)} observations')

In [ ]:
# Show a random example (run this cell again to see a different entry)
search_results.sample(1).T

In [ ]:
# Show the first and last five entries.
search_results

## Select observation

A `sequence_id` is used to explore observations. Here we take the most recent observation to get the `sequence_id`.

In [ ]:
# We choose an entry that only has a few images while running on binder and get the most recent.
record = search_results.query('num_images < 15').iloc[-1]
record.T

## Get Observation info

We can get more information about the observation by creating an `ObservationInfo` object. You can create it with just a `sequence_id` (e.g. `ObservationInfo(sequence_id=record.sequence_id)) but here we pass all the metadata.

In [ ]:
obs_info = ObservationInfo(meta=record)

# Show the observation object.
obs_info

In [ ]:
obs_info.meta

### Show example of the image metadata and how to save to file

The metadata for each image is stored in the `obs_info.image_metadata` dataframe. 

We view a random example and then write the metadata to a file that can be downloaded in the file browser on the left.

In [ ]:
obs_info.image_metadata

In [ ]:
# Randomly sample one image from the dataframe then transpose it so it's easier to read.
obs_info.image_metadata.sample(1).T

In [ ]:
# obs_info.image_metadata.to_csv(f'{obs_info.sequence_id}-image-metadata.csv')

### Download files locally

We can use the built-in `download_images` method of our `obs_info` object to download the files locally.

By default this will create an output directory that is the same as the `sequence_id` but this can be changed. You can also pass a limited list of files for download. 

Run `obs_info.download_images?` for more information.

In [ ]:
# image_files = obs_info.download_images()

### Write image list to a CSV file, use `wget` to download

Or, if you prefer you can just get a list of files saved as csv and work on them in a command line terminal.

This first creates a DataFrame then immediately writes that to a CSV file. There are many ways to accomplish this but this is an easy way.

We then view the last few lines of the file using the command line tool `tail` called in a cell with the `!` operator.

We then use the same `!` operator to call `wget` in the shell and download a list of files. Note that you can just copy the CSV file to your computer or server and use the `wget -i <filename.txt>` command to download all the files. **Note that this can be a lot of data!**

In [ ]:
# Create a dataframe and then write it to a CSV file.
pd.DataFrame(obs_info.get_image_list()).to_csv(f'{obs_info.sequence_id}-raw.txt', index=False, header=False)

In [ ]:
# View the last entries in the file.
!tail {obs_info.sequence_id}-raw.txt

#### Download the data

In [ ]:
### Uncomment to run ###
# !wget -i {obs_info.sequence_id}-raw.txt
